# FCUL ALS Model Interpretability
---

Exploring the ALS dataset from Faculdade de Ciências da Universidade de Lisboa (FCUL) with the data from over 1000 patients collected in Portugal.

Using different interpretability approaches so as to understand the outputs of the models trained on FCUL's ALS dataset.

## Importing the necessary packages

In [ ]:
import os                                  # os handles directory/workspace changes
import torch                               # PyTorch to create and apply deep learning models
import xgboost as xgb                      # Gradient boosting trees models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import joblib                              # Save and load scikit-learn models in disk
import pickle                              # Save python objects in files
import yaml                                # Save and load YAML files
from datetime import datetime              # datetime to use proper date and time formats
from ipywidgets import interact            # Display selectors and sliders
import shap                                # Model-agnostic interpretability package inspired on Shapley values
import plotly.graph_objs as go             # Plotly for interactive and pretty plots
from model_interpreter.model_interpreter import ModelInterpreter # Class that enables the interpretation of models that handle variable sequence length input data

In [ ]:
import pixiedust                           # Debugging in Jupyter Notebook cells

In [ ]:
# Path to the parquet dataset files
data_path = 'Datasets/Thesis/FCUL_ALS/cleaned/'
# Path to the data + SHAP values dataframes
data_n_shap_path = 'Datasets/Thesis/FCUL_ALS/interpreted/'
# Path to the code files
project_path = 'GitHub/FCUL_ALS_Disease_Progression/'
# Path to the models
models_path = f'{project_path}models/'
# Path to the model interpreters
interpreters_path = f'{project_path}interpreters/'

In [ ]:
# Change to the scripts directory
os.chdir("../scripts/")
import utils                               # Context specific (in this case, for the ALS data) methods
import Models                              # Deep learning models
# Change to parent directory (presumably "Documents")
os.chdir("../../..")
import pandas as pd                        # Pandas to load and handle the data
import data_utils as du                    # Data science and machine learning relevant methods

In [ ]:
du.set_pandas_library(lib='pandas')

Allow pandas to show more columns:

In [ ]:
pd.set_option('display.max_columns', 3000)
pd.set_option('display.max_rows', 3000)

Set the random seed for reproducibility:

In [ ]:
du.set_random_seed(42)

## Initializing variables

Dataset parameters:

In [ ]:
id_column = 'subject_id'                   # Name of the sequence ID column
ts_column = 'ts'                           # Name of the timestamp column
label_column = 'niv_label'                 # Name of the label column
padding_value = 999999                     # Padding value used to fill in sequences up to the maximum sequence length

One hot encoding columns categorization:

In [ ]:
stream_categ_feat_ohe = open(f'{data_path}categ_feat_ohe.yml', 'r')

In [ ]:
categ_feat_ohe = yaml.load(stream_categ_feat_ohe, Loader=yaml.FullLoader)
categ_feat_ohe

In [ ]:
list(categ_feat_ohe.keys())

Dataloaders parameters:

In [ ]:
test_train_ratio = 0.25                    # Percentage of the data which will be used as a test set
validation_ratio = 0.1                     # Percentage of the data from the training set which is used for validation purposes
batch_size = 32                            # Number of unit stays in a mini batch

Model to interpret:

In [ ]:
model_filename = None                      # Name of the file containing the model that will be loaded
model_class = None                         # Python class name that corresponds to the chosen model's type
model = None                               # Machine learning model object
dataset_mode = 'one hot encoded'           # The mode in which we'll use the data, either one hot encoded or pre-embedded
ml_core = 'deep learning'                  # The core machine learning type we'll use; either traditional ML or DL
use_delta_ts = False                       # Indicates if we'll use time variation info
time_window_days = 90                      # Number of days on which we want to predict NIV
is_custom = False                          # Indicates if the model being used is a custom built one
@interact
def get_dataset_mode(model_name=['Bidirectional LSTM with embedding layer',
                                 'Bidirectional RNN with embedding layer and delta_ts',
                                 'Bidirectional LSTM with delta_ts',
                                 'Regular LSTM',
                                 'MF1-LSTM',
                                 'XGBoost',
                                 'Logistic regression']):
    global model_filename, model_class, model, dataset_mode, ml_core, use_delta_ts, time_window_days, is_custom
    if model_name == 'Bidirectional LSTM with embedding layer':
        # Set the model file and class names, then load the model
        model_filename = 'lstm_bidir_pre_embedded_90dayswindow_0.2490valloss_06_07_2020_03_47.pth'
        model_class = 'VanillaLSTM'
        model = du.deep_learning.load_checkpoint(f'{models_path}{model_filename}', getattr(Models, model_class))
        # Set the use of an embedding layer
        dataset_mode = 'pre-embedded'
        # Set it as a custom model
        is_custom = True
    elif model_name == 'Bidirectional RNN with embedding layer and delta_ts':
        # Set the model file and class names, then load the model
        model_filename = 'rnn_bidir_pre_embedded_delta_ts_90dayswindow_0.3059valloss_06_07_2020_03_10.pth'
        model_class = 'VanillaRNN'
        model = du.deep_learning.load_checkpoint(f'{models_path}{model_filename}', getattr(Models, model_class))
        # Set the use of an embedding layer
        dataset_mode = 'pre-embedded'
        # Set the use of delta_ts
        use_delta_ts = 'normalized'
        # Set it as a custom model
        is_custom = True
    elif model_name == 'Bidirectional LSTM with delta_ts':
        # Set the model file and class names, then load the model
        model_filename = 'lstm_bidir_pre_embedded_delta_ts_90dayswindow_0.3481valloss_06_07_2020_04_15.pth'
        model_class = 'VanillaLSTM'
        model = du.deep_learning.load_checkpoint(f'{models_path}{model_filename}', getattr(Models, model_class))
        # Set the use of delta_ts
        use_delta_ts = 'normalized'
        # Set it as a custom model
        is_custom = True
    elif model_name == 'Regular LSTM':
        # Set the model file and class names, then load the model
        model_filename = 'lstm_one_hot_encoded_90dayswindow_0.4363valloss_06_07_2020_03_28.pth'
        model_class = 'VanillaLSTM'
        model = du.deep_learning.load_checkpoint(f'{models_path}{model_filename}', getattr(Models, model_class))
        # Set it as a custom model
        is_custom = True
    elif model_name == 'MF1-LSTM':
        # Set the model file and class names, then load the model
        model_filename = 'mf1lstm_one_hot_encoded_90dayswindow_0.6009valloss_07_07_2020_03_46.pth'
        model_class = 'MF1LSTM'
        model = du.deep_learning.load_checkpoint(f'{models_path}{model_filename}', getattr(Models, model_class))
        # Set the use of delta_ts
        use_delta_ts = 'normalized'
        # Set it as a custom model
        is_custom = True
    elif model_name == 'XGBoost':
        # Set the model file and class names, then load the model
        model_filename = 'xgb_0.5926valloss_09_07_2020_02_40.pth'
        model_class = 'XGBoost'
        model = xgb.XGBClassifier()
        model.load_model(f'{models_path}{model_filename}')
        # Set as a traditional ML model
        ml_core = 'machine learning'
    elif model_name == 'Logistic regression':
        # Set the model file and class names, then load the model
        model_filename = 'logreg_0.6210valloss_09_07_2020_02_54.pth'
        model_class = 'logreg'
        model = joblib.load(f'{models_path}{model_filename}')
        # Set as a traditional ML model
        ml_core = 'machine learning'
    print(model)

## Loading the data

In [ ]:
ALS_df = pd.read_csv(f'{data_path}FCUL_ALS_cleaned.csv')
ALS_df.head()

Remove the `Unnamed: 0` column:

In [ ]:
ALS_df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
ALS_df.columns

In [ ]:
len(ALS_df.columns)

Find the maximum sequence length, so that the ML models and their related methods can handle all sequences, which have varying sequence lengths:

In [ ]:
total_length = ALS_df.groupby(id_column)[ts_column].count().max()
total_length

## Preprocessing data

Define the label column, in case we're using a time window different than 90 days:

In [ ]:
if time_window_days is not 90:
    # Recalculate the NIV label, based on the chosen time window
    ALS_df[label_column] = utils.set_niv_label(ALS_df, time_window_days)
    display(ALS_df.head())

Remove the `niv` column:

In [ ]:
ALS_df.drop(columns=['niv'], inplace=True)

Add the `delta_ts` (time variation between samples) if required:

In [ ]:
if use_delta_ts is not False:
    # Create a time variation column
    ALS_df['delta_ts'] = ALS_df.groupby(id_column).ts.diff()
    # Fill all the delta_ts missing values (the first value in a time series) with zeros
    ALS_df['delta_ts'] = ALS_df['delta_ts'].fillna(0)
if use_delta_ts == 'normalized':
    # Normalize the time variation data
    # NOTE: When using the MF2-LSTM model, since it assumes that the time
    # variation is in days, we shouldn't normalize `delta_ts` with this model.
    ALS_df['delta_ts'] = (ALS_df['delta_ts'] - ALS_df['delta_ts'].mean()) / ALS_df['delta_ts'].std()
if use_delta_ts is not False:
    display(ALS_df.head())

Convert into a padded tensor:

In [ ]:
data = du.padding.dataframe_to_padded_tensor(ALS_df, padding_value=padding_value,
                                             label_column=label_column, inplace=True)
data

Set the embedding configuration, if needed:

In [ ]:
# Indices of the ID, timestamp and label columns
id_column_idx = du.search_explore.find_col_idx(ALS_df, id_column)
ts_column_idx = du.search_explore.find_col_idx(ALS_df, ts_column)
label_column_idx = du.search_explore.find_col_idx(ALS_df, label_column)
print(
f'''ID index: {id_column_idx}
Timestamp index: {ts_column_idx}
Label index: {label_column_idx}'''
)

In [ ]:
if dataset_mode == 'one hot encoded':
    embed_features = None
else:
    embed_features = list()
    if len(categ_feat_ohe.keys()) == 1:
        for ohe_feature in list(categ_feat_ohe.values())[0]:
            # Find the current feature's index so as to be able to use it as a tensor
            feature_idx = du.search_explore.find_col_idx(ALS_df, ohe_feature)
            # Decrease the index number if it's larger than the label column (which will be removed)
            if feature_idx > label_column_idx:
                feature_idx = feature_idx - 1
            embed_features.append(feature_idx)
    else:
        for i in range(len(categ_feat_ohe.keys())):
            tmp_list = list()
            for ohe_feature in list(categ_feat_ohe.values())[i]:
                # Find the current feature's index so as to be able to use it as a tensor
                feature_idx = du.search_explore.find_col_idx(ALS_df, ohe_feature)
                # Decrease the index number if it's larger than the label column (which will be removed)
                if feature_idx > label_column_idx:
                    feature_idx = feature_idx - 1
                tmp_list.append(feature_idx)
            # Add the current feature's list of one hot encoded columns
            embed_features.append(tmp_list)
print(f'Embedding features: {embed_features}')

## Defining the dataset object

In [ ]:
dataset = du.datasets.Time_Series_Dataset(ALS_df, data, padding_value=padding_value,
                                          label_name=label_column)

In [ ]:
dataset.__len__()

## Separating into train and validation sets

In [ ]:
(train_dataloader, val_dataloader, test_dataloader,
train_indeces, val_indeces, test_indeces) = du.machine_learning.create_train_sets(dataset,
                                                                                  test_train_ratio=test_train_ratio,
                                                                                  validation_ratio=validation_ratio,
                                                                                  batch_size=batch_size,
                                                                                  get_indices=True)

In [ ]:
# Get the full arrays of each set
train_features, train_labels = dataset.X[train_indeces], dataset.y[train_indeces]
val_features, val_labels = dataset.X[val_indeces], dataset.y[val_indeces]
test_features, test_labels = dataset.X[test_indeces], dataset.y[test_indeces]

In [ ]:
# Ignore the dataloaders, we only care about the full arrays when using scikit-learn or XGBoost
del train_dataloader
del val_dataloader
del test_dataloader

In [ ]:
if ml_core == 'machine learning':
    # Remove the ID and timestamp columns from the data arrays
    train_features = train_features[:, :, 2:]
    val_features = val_features[:, :, 2:]
    test_features = test_features[:, :, 2:]
    # Reshape the data into a 2D format
    train_features = train_features.reshape(-1, train_features.shape[-1])
    val_features = val_features.reshape(-1, val_features.shape[-1])
    test_features = test_features.reshape(-1, test_features.shape[-1])
    train_labels = train_labels.reshape(-1)
    val_labels = val_labels.reshape(-1)
    test_labels = test_labels.reshape(-1)
    # Remove padding samples from the data
    train_features = train_features[[padding_value not in row for row in train_features]]
    val_features = val_features[[padding_value not in row for row in val_features]]
    test_features = test_features[[padding_value not in row for row in test_features]]
    train_labels = train_labels[[padding_value not in row for row in train_labels]]
    val_labels = val_labels[[padding_value not in row for row in val_labels]]
    test_labels = test_labels[[padding_value not in row for row in test_labels]]
    # Convert from PyTorch tensor to NumPy array
    train_features = train_features.numpy()
    val_features = val_features.numpy()
    test_features = test_features.numpy()
    train_labels = train_labels.numpy()
    val_labels = val_labels.numpy()
    test_labels = test_labels.numpy()

## Interpreting the model

In [ ]:
test_features.shape

Define the interpreter:

In [ ]:
if ml_core == 'deep learning':
    # Calculating the number of times to re-evaluate the model when explaining each prediction,
    # based on SHAP's formula of nsamples = 2 * n_features + 2048
    SHAP_bkgnd_samples = 2 * test_features.shape[-1] + 2048
    print(SHAP_bkgnd_samples)

In [ ]:
if ml_core == 'deep learning':
    interpreter = ModelInterpreter(model, ALS_df, model_type='multivariate_rnn', id_column=0, 
                                   inst_column=1, fast_calc=True, SHAP_bkgnd_samples=SHAP_bkgnd_samples,
                                   random_seed=du.random_seed, padding_value=padding_value,
                                   is_custom=is_custom, total_length=total_length)
# else:

Calculate the feature importance scores (through SHAP values):

In [ ]:
if ml_core == 'deep learning':
    _ = interpreter.interpret_model(test_data=test_features[:5],
                                    test_labels=test_labels[:5],
                                    instance_importance=False, 
                                    feature_importance='shap')
# else:

In [ ]:
if ml_core == 'deep learning':
    print(interpreter.explainer.expected_value)
else:
    print(explainer.expected_value)

## Saving a dataframe with the resulting SHAP values

In [ ]:
interpreter.feat_scores.shape

In [ ]:
du.visualization.shap_summary_plot(interpreter.feat_scores, interpreter.feat_names, max_display=15)

In [ ]:
du.visualization.shap_waterfall_plot(interpreter.explainer.expected_value[0], interpreter.feat_scores[0, 2, :], 
                                     interpreter.test_data[0, 2, 2:], interpreter.feat_names)

In [ ]:
if ml_core == 'deep learning':
    data_n_shap_df = interpreter.shap_values_df()
    data_n_shap_df.head()
# else:

In [ ]:
data_n_shap_df.to_csv(f'{data_n_shap_path}fcul_als_with_shap_for_{model_filename}.csv')